# Importamos las librerias necesarias# Torre de Hanoi - Algoritmo A*
# Introducción a la Inteligencia Artificial

## Problema de la Torre de Hanoi con 5 discos

### Descripción del problema
La Torre de Hanoi es un problema clásico que consiste en mover una torre de discos de diferentes tamaños de una varilla a otra, siguiendo estas reglas:
1. Solo se puede mover un disco a la vez
2. Un disco solo puede colocarse sobre otro disco más grande
3. Objetivo: mover todos los discos de la varilla izquierda (A) a la derecha (C)

### Estado inicial: Todos los discos en la varilla A
### Estado objetivo: Todos los discos en la varilla C

# Importamos las librerias necesarias

In [ ]:
import heapq
import time
import json
from typing import List, Tuple, Optional, Dict, Any
from datetime import datetime
import tracemalloc

# Definimos las clases necesarias

In [ ]:
class EstadoHanoi:
    """
    Representa un estado del problema de la Torre de Hanoi.
    Cada estado contiene las tres varillas con sus respectivos discos.
    """

    def __init__(self, varillas: List[List[int]]):
        self.varillas = [list(varilla) for varilla in varillas]  # Copia profunda

    def __eq__(self, other):
        return self.varillas == other.varillas

    def __hash__(self):
        return hash(tuple(tuple(varilla) for varilla in self.varillas))

    def __str__(self):
        return f"A:{self.varillas[0]} B:{self.varillas[1]} C:{self.varillas[2]}"

    def es_valido(self) -> bool:
        """Verifica si el estado actual es válido según las reglas de Hanoi"""
        for varilla in self.varillas:
            for i in range(len(varilla) - 1):
                if varilla[i] > varilla[i + 1]:  # Disco más grande sobre uno más pequeño
                    return False
        return True

    def obtener_movimientos_posibles(self) -> List['EstadoHanoi']:
        """Genera todos los estados posibles desde el estado actual"""
        movimientos = []

        for origen in range(3):
            if self.varillas[origen]:  # Si la varilla tiene discos
                disco = self.varillas[origen][-1]  # Disco superior

                for destino in range(3):
                    if origen != destino:
                        # Verificar si el movimiento es válido
                        if not self.varillas[destino] or self.varillas[destino][-1] > disco:
                            nuevo_estado = EstadoHanoi(self.varillas)
                            nuevo_estado.varillas[origen].pop()
                            nuevo_estado.varillas[destino].append(disco)
                            movimientos.append(nuevo_estado)

        return movimientos

In [ ]:
class NodoAEstrella:
    """
    Nodo para el algoritmo A*.
    Contiene el estado, costos g y h, y referencia al nodo padre.
    """

    def __init__(self, estado: EstadoHanoi, g: int = 0, h: int = 0, padre: Optional['NodoAEstrella'] = None):
        self.estado = estado
        self.g = g  # Costo desde el inicio
        self.h = h  # Heurística
        self.f = g + h  # Función de evaluación
        self.padre = padre

    def __lt__(self, other):
        return self.f < other.f

In [ ]:
class TorreHanoiAStar:
    """
    Implementación del algoritmo A* para resolver la Torre de Hanoi.
    """

    def __init__(self, num_discos: int = 5):
        self.num_discos = num_discos
        self.estado_inicial = EstadoHanoi([list(range(num_discos, 0, -1)), [], []])
        self.estado_objetivo = EstadoHanoi([[], [], list(range(num_discos, 0, -1))])
        self.nodos_explorados = 0
        self.nodos_expandidos = 0

    def heuristica_creativa(self, estado: EstadoHanoi) -> int:
        """
        Heurística creativa: Combinación de múltiples factores

        Fundamentación:
        1. Discos mal ubicados: Penaliza discos que no están en la varilla objetivo
        2. Discos bloqueados: Penaliza discos que tienen otros encima en varillas incorrectas
        3. Orden incorrecto: Penaliza cuando los discos no están en orden correcto
        4. Distancia de movimiento: Considera qué tan lejos están los discos de su destino

        Esta heurística es admisible porque nunca sobreestima el costo real,
        ya que cada factor representa movimientos mínimos necesarios.
        """
        h = 0

        # Factor 1: Discos que no están en la varilla objetivo (C)
        discos_mal_ubicados = 0
        for i in range(2):  # Varillas A y B
            discos_mal_ubicados += len(estado.varillas[i])

        # Factor 2: Discos bloqueados en varillas incorrectas
        discos_bloqueados = 0
        for i in range(2):  # Solo varillas A y B
            varilla = estado.varillas[i]
            for j, disco in enumerate(varilla):
                # Si hay discos encima, está bloqueado
                if j < len(varilla) - 1:
                    discos_bloqueados += 1

        # Factor 3: Verificar orden en varilla objetivo
        orden_incorrecto = 0
        varilla_c = estado.varillas[2]
        for i in range(len(varilla_c)):
            if varilla_c[i] != self.num_discos - i:
                orden_incorrecto += 1

        # Factor 4: Peso por tamaño de disco (discos más grandes son más costosos de mover)
        peso_discos = 0
        for i in range(2):
            for disco in estado.varillas[i]:
                peso_discos += disco * 0.1  # Pequeño peso adicional

        # Combinación de factores
        h = discos_mal_ubicados * 2 + discos_bloqueados + orden_incorrecto + peso_discos

        return int(h)

    def heuristica_simple(self, estado: EstadoHanoi) -> int:
        """
        Heurística simple propuesta: -1 por cada disco en posición correcta
        (Convertida a positiva para A*)
        """
        discos_correctos = len(estado.varillas[2])  # Discos en varilla C
        return self.num_discos - discos_correctos

    def a_estrella(self, usar_heuristica_creativa: bool = True) -> Tuple[List[EstadoHanoi], dict]:
        """
        Implementación del algoritmo A*.

        Returns:
            - Lista de estados que forman la solución
            - Diccionario con estadísticas del algoritmo
        """
        print(f"Iniciando A* con heurística {'creativa' if usar_heuristica_creativa else 'simple'}...")
        print(f"Estado inicial: {self.estado_inicial}")
        print(f"Estado objetivo: {self.estado_objetivo}")
        print("-" * 60)

        inicio_tiempo = time.time()

        # Inicialización
        heuristica = self.heuristica_creativa if usar_heuristica_creativa else self.heuristica_simple
        nodo_inicial = NodoAEstrella(self.estado_inicial, 0, heuristica(self.estado_inicial))

        lista_abierta = [nodo_inicial]
        lista_cerrada = set()
        self.nodos_explorados = 0
        self.nodos_expandidos = 0

        while lista_abierta:
            # Seleccionar nodo con menor f
            nodo_actual = heapq.heappop(lista_abierta)
            self.nodos_explorados += 1

            # Verificar si alcanzamos el objetivo
            if nodo_actual.estado == self.estado_objetivo:
                tiempo_total = time.time() - inicio_tiempo

                # Reconstruir camino
                camino = []
                nodo = nodo_actual
                while nodo:
                    camino.append(nodo.estado)
                    nodo = nodo.padre
                camino.reverse()

                estadisticas = {
                    'movimientos': len(camino) - 1,
                    'nodos_explorados': self.nodos_explorados,
                    'nodos_expandidos': self.nodos_expandidos,
                    'tiempo_ejecucion': tiempo_total,
                    'costo_solucion': nodo_actual.g,
                    'heuristica_usada': 'creativa' if usar_heuristica_creativa else 'simple'
                }

                return camino, estadisticas

            # Agregar a lista cerrada
            lista_cerrada.add(nodo_actual.estado)

            # Expandir nodos sucesores
            self.nodos_expandidos += 1
            for estado_sucesor in nodo_actual.estado.obtener_movimientos_posibles():
                if estado_sucesor not in lista_cerrada:
                    g_nuevo = nodo_actual.g + 1
                    h_nuevo = heuristica(estado_sucesor)
                    nodo_sucesor = NodoAEstrella(estado_sucesor, g_nuevo, h_nuevo, nodo_actual)

                    # Verificar si ya está en lista abierta con mejor costo
                    en_abierta = False
                    for i, nodo in enumerate(lista_abierta):
                        if nodo.estado == estado_sucesor:
                            if nodo_sucesor.f < nodo.f:
                                lista_abierta[i] = nodo_sucesor
                                heapq.heapify(lista_abierta)
                            en_abierta = True
                            break

                    if not en_abierta:
                        heapq.heappush(lista_abierta, nodo_sucesor)

        return None, {'error': 'No se encontró solución'}